<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/age_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies


In [0]:
!pip install -U -q PyDrive

## Data Preparation

### Download Data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

utk_images_7z_id = ''
utk_landmarks_7z_id = ''

appareal_labels_json_id = ''
appareal_images_7z_id = ''
appareal_landmarks_7z_id = ''

wiki_labels_json_id = ''
wiki_images_7z_id = ''
wiki_landmarks_7z_id = ''

if not os.path.exists('./data'):
  os.makedirs('./data')
if not os.path.exists('./data/utk'):
  os.makedirs('./data/utk')
if not os.path.exists('./data/appareal'):
  os.makedirs('./data/appareal')
if not os.path.exists('./data/wiki'):
  os.makedirs('./data/wiki')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
  
print('downloading utk data...')
drive.CreateFile({ 'id': utk_images_7z_id }).GetContentFile('./data/utk/images.7z')
drive.CreateFile({ 'id': utk_landmarks_7z_id }).GetContentFile('./data/utk/landmarks.7z')

print('downloading appareal data...')
drive.CreateFile({ 'id': appareal_labels_json_id }).GetContentFile('./data/appareal/labels.json')
drive.CreateFile({ 'id': appareal_images_7z_id }).GetContentFile('./data/appareal/images.7z')
drive.CreateFile({ 'id': appareal_landmarks_7z_id }).GetContentFile('./data/appareal/landmarks.7z')

print('downloading wiki data...')
drive.CreateFile({ 'id': wiki_labels_json_id }).GetContentFile('./data/wiki/labels.json')
drive.CreateFile({ 'id': wiki_images_7z_id }).GetContentFile('./data/wiki/images.7z')
drive.CreateFile({ 'id': wiki_landmarks_7z_id }).GetContentFile('./data/wiki/landmarks.7z')
  
print('done!')

### Unzip Data

In [0]:
!cd ./data/utk && p7zip -d ./images.7z
!cd ./data/utk && p7zip -d ./landmarks.7z
!cd ./data/appareal && p7zip -d ./images.7z
!cd ./data/appareal && p7zip -d ./landmarks.7z
!cd ./data/wiki && p7zip -d ./images.7z
!cd ./data/wiki && p7zip -d ./landmarks.7z

## Training

### Preprocessing

In [0]:
import cv2
import math
import json
import numpy as np
from random import randint

def num_in_range(val, min_val, max_val):
  return min(max(min_val, val), max_val)

def random_crop(img, landmarks):
  height, width, _ = img.shape
  min_x, min_y, max_x, max_y = width, height, 0, 0
  for pt in landmarks:
    min_x = pt['x'] if pt['x'] < min_x else min_x
    min_y = pt['y'] if pt['y'] < min_y else min_y
    max_x = max_x if pt['x'] < max_x else pt['x']
    max_y = max_y if pt['y'] < max_y else pt['y']
  
  min_x = int(num_in_range(min_x, 0, 1) * width)
  min_y = int(num_in_range(min_y, 0, 1) * height)
  max_x = int(num_in_range(max_x, 0, 1) * width)
  max_y = int(num_in_range(max_y, 0, 1) * height)
  x0 = randint(0, min_x)
  y0 = randint(0, min_y)
  x1 = randint(0, abs(width - max_x)) + max_x
  y1 = randint(0, abs(height - max_y)) + max_y

  return img[y0:y1, x0:x1]

def resize_preserve_aspect_ratio(img, size):
  height, width, _ = img.shape
  max_dim = max(height, width)
  ratio = size / float(max_dim)
  shape = (height * ratio, width * ratio)
  resized_img = cv2.resize(img, (int(round(height * ratio)), int(round(width * ratio))))
  
  return resized_img
  
def pad_to_square(img):
  height, width, channels = img.shape
  max_dim = max(height, width)
  square_img = np.zeros([max_dim, max_dim, channels])

  dx = math.floor(abs(max_dim - width) / 2)
  dy = math.floor(abs(max_dim - height) / 2)
  square_img[dy:dy + height,dx:dx + width] = img

  return square_img

def preprocess(img, landmarks, size):
  cropped_img = random_crop(img, landmarks)
  resized_img = resize_preserve_aspect_ratio(cropped_img, size)
  square_img = pad_to_square(resized_img)
  
  return square_img

def load_batch(datas):
  preprocessed_imgs = []
  
  for data in datas:
    db = data['db']
    img_file = data['file']
    file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
    landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
    img_file_path = './data/' + db + '/cropped-images/' + img_file
    landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file
    
    img = cv2.imread(img_file_path)
    with open(landmarks_file_path) as json_file:  
      landmarks = json.load(json_file)
      preprocessed_img = preprocess(img, landmarks, 112)
      preprocessed_imgs.append(preprocessed_img)
      
  return np.stack(preprocessed_imgs, axis=0)
  

### Neural Network

In [0]:
import tensorflow as tf

def conv2d(x, weights, stride):
  out = tf.nn.conv2d(x, weights['filter'], stride, 'same')
  out = tf.add(out, weights['bias'])
  return out

def depthwise_separable_conv2d(x, weights, stride):
  out = tf.nn.separable_conv2d(x, params['depthwise_filter'], params['pointwise_filter'], stride, 'same')
  out = tf.add(out, weights['bias'])
  return out
  
def fully_connected(x, weights):
  out = tf.reshape(x, [-1, weights['weights'].get_shape().as_list()[0]])
  out = tf.matmul(out, weights['weights'])
  out = tf.add(out, weights['bias'])
  return out

def dense_block(x, weights, is_first_layer = False, is_scale_down = True):
  initial_stride = [2, 2]  if is_scale_down else [1, 1]
  out1 = conv2d(x, weights['conv0'], initial_stride) if is_first_layer else depthwise_separable_conv2d(x, weights['conv0'], initial_stride)
  
  in2 = tf.nn.relu(out1)
  out2 = depthwise_separable_conv2d(in2, weights['conv1'], [1, 1])

  in3 = tf.nn.relu(tf.add(out1, out2))
  out3 = depthwise_separable_conv2d(in3, weights['conv2'], [1, 1])

  in4 = tf.nn.relu(tf.add(out1, tf.add(out2, out3)))
  out4 = depthwise_separable_conv2d(in4, weights['conv3'], [1, 1])

  return tf.nn.relu(tf.add(out1, tf.add(out2, tf.add(out3, out4))))


def forward(batch_tensor, weights):
  mean_rgb = [122.782, 117.001, 104.298]
  normalized = tf.div(normalize(batch_tensor, mean_rgb), tf.scalar(255))

  out = dense_block(normalized, weights['dense0'], true)
  out = dense_block(out, weights['dense1'])
  out = dense_block(out, weights['dense2'])
  out = dense_block(out, weights['dense3'])
  out = tf.nn.avg_pool(out, [7, 7], [2, 2], 'valid')
  out = fully_connected(out, weights['fc_age'])

  return out



### Weight Serialization

In [0]:
class WeightProcessor:
  def __init__(self, process_weights, processor_bias):
    self.process_weights = process_weights
    self.processor_bias = processor_bias
  
  def process_conv_weights(self, channels_in, channels_out, prefix):
    self.process_weights([3, 3, channels_in, channels_out], prefix + '/filter')
    self.processor_bias([channels_out], prefix + '/bias')

  def process_depthwise_separable_conv2d_weights(self, channels_in, channels_out, prefix):
    self.process_weights([3, 3, channels_in, 1], prefix + '/depthwise_filter'),
    self.process_weights([1, 1, channels_in, channels_out], prefix + '/pointwise_filter'),
    self.processor_bias([channels_out], prefix + '/bias')

  def process_dense_block_weights(self, channels_in, channels_out, prefix, is_first_layer = False):
    conv0_processor = self.process_conv_weights if is_first_layer else self.process_depthwise_separable_conv2d_weights
    conv0_processor(channels_in, channels_out, prefix + '/conv0')
    self.process_depthwise_separable_conv2d_weights(channels_in, channels_out, prefix + '/conv1')
    self.process_depthwise_separable_conv2d_weights(channels_in, channels_out, prefix + '/conv2')
    self.process_depthwise_separable_conv2d_weights(channels_in, channels_out, prefix + '/conv3')

  def process(self):
    self.process_dense_block_weights(3, 32, 'dense0', True)
    self.process_dense_block_weights(32, 64, 'dense1')
    self.process_dense_block_weights(64, 128, 'dense2')
    self.process_dense_block_weights(128, 256, 'dense3')
    self.process_weights([128, 1], 'fc_age/weights')
    self.processor_bias([1], 'fc_age/bias')

def build_weight_map(tensors, tensor_paths):
  weights = {}
  for idx, tensor in enumerate(tensors):
    tensor_path = tensor_paths[idx]
    
    tmp = weights
    keys = tensor_path.split('/')
    for path_idx, key in enumerate(keys):
      is_end = path_idx == len(keys) - 1
      tmp[key] = tensor if is_end else (tmp[key] if key in tmp else {})
      tmp = tmp[key]
      
  return weights
  
    
def init_weights(weight_initializer = tf.keras.initializers.glorot_normal(), bias_initializer = tf.zeros):
  tensors = []
  tensor_paths = []
  def process_weights(shape, tensor_path):
    tensors.append(weight_initializer(shape))
    tensor_paths.append(tensor_path)
  def process_bias(shape, tensor_path):
    tensors.append(bias_initializer(shape))
    tensor_paths.append(tensor_path)

  WeightProcessor(process_weights, process_bias).process()

  return build_weight_map(tensors, tensor_paths)
  
def load_weights(checkpoint_file):  
  checkpoint_data = np.load(checkpoint_file)
  
  idx = 0
  tensors = []
  tensor_paths = []
  def extract_weights_from_shape(shape, tensor_path):
    nonlocal idx
    size = 1
    for val in shape:
      size = size * val
    tensor = tf.convert_to_tensor(np.reshape(checkpoint_data[idx:idx + size], shape), dtype=tf.float32)
    
    idx += size
    tensors.append(tensor)
    tensor_paths.append(tensor_path)

  WeightProcessor(extract_weights_from_shape, extract_weights_from_shape).process()

  return build_weight_map(tensors, tensor_paths)

def save_weights(checkpoint_file, weights):  
  checkpoint_data = np.array([])
  def append_weights(shape, tensor_path):
    nonlocal checkpoint_data
    tmp = weights
    for key in tensor_path.split('/'):
      tmp = tmp[key]
    tensor_data_flat = tmp.eval().flatten()
    checkpoint_data = np.append(checkpoint_data, tensor_data_flat)
    
  WeightProcessor(append_weights, append_weights).process()
  np.save(checkpoint_file, checkpoint_data)

### Training